In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
batch_size_pipeline = 4
num_steps_training = 500
learning_rate_pipeline = 0.08/100
warmup_learning_rate_pipeline = 0.026666/100
PRETRAINED_MODEL_PATH = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

# 0. Setup Paths

In [ ]:
import datetime
import pytz

def shorten_time(date):
    dt_year = date[0:4]
    dt_month = date[5:7]
    dt_day = date[8:10]
    dt_hr = date[11:13]
    dt_min = date[14:16]
    dt_sec = date[17:19]
    dt_full = "{}{}{}_{}{}{}".format(dt_year,dt_month,dt_day,dt_hr,dt_min,dt_sec)
    return dt_full

start_date = str(datetime.datetime.now(pytz.timezone('Asia/Bangkok')))[:19]
start_date_short = shorten_time(start_date)
print(start_date)
print(start_date_short)

In [ ]:
gg_drive_path = "/content/drive/MyDrive/"

In [ ]:
models_drive_path = "{}workspace_{}".format(gg_drive_path,start_date_short)
print(models_drive_path)

In [ ]:
import os

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_custom'
PRETRAINED_MODEL_NAME = PRETRAINED_MODEL_PATH[PRETRAINED_MODEL_PATH.rfind("/")+1:-7]
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc'),
    'MODEL_PATH': os.path.join(models_drive_path),
    'CHECKPOINT_PATH': os.path.join(models_drive_path,CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join(models_drive_path,CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join(models_drive_path,CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join(models_drive_path,CUSTOM_MODEL_NAME, 'tfliteexport'), 
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join(models_drive_path, CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 1. Install TF Object Dectection API

In [ ]:
# https://www.tensorflow.org/install/source_windows

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

In [ ]:
print(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py'))

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
import object_detection

# 2. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_PATH}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_PATH)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

# 3. Create Label Map

In [ ]:
labels = [{'name':'car', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 4. Create TF records

In [ ]:
# Unrar image files from drive and locate in /content/Tensorflow/workspace/images
!unrar x /content/drive/MyDrive/car_coco_2017.rar -d /content/Tensorflow/workspace/images

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

# 5. Copy Model Config to Training Folder

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

In [ ]:
print(os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config'))

In [ ]:
print(os.path.join(paths['CHECKPOINT_PATH']))

# 6. Update Config For Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = batch_size_pipeline
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = learning_rate_pipeline
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = warmup_learning_rate_pipeline
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 7. Train the model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'],num_steps_training)

In [ ]:
print(command)

In [ ]:
!{command}

# 8. Freezing the Graph

In [ ]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [ ]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}

# 9. Zip model and copy to GG Drive

In [ ]:
!cp -r {paths['OUTPUT_PATH']} {paths['MODEL_PATH']}

In [ ]:
os.rename("{}/{}".format(paths['MODEL_PATH'],"export"),"{}/model_{}".format(paths['MODEL_PATH'],start_date_short))

In [ ]:
%cd {paths['MODEL_PATH']}
!ls

In [ ]:
export_model_path = "model_{}".format(start_date_short)
export_model_zip = "{}.zip".format(export_model_path)
print(export_model_path)
print(export_model_zip)

In [ ]:
!zip -r {export_model_zip} {export_model_path}

# 10. Notification via Email

In [ ]:
import datetime
import pytz

finish_date = str(datetime.datetime.now(pytz.timezone('Asia/Bangkok')))[:19]
finish_date_short = shorten_time(finish_date)

print(finish_date)
print(finish_date_short)

In [ ]:
sd_year = int(start_date_short[0:4])
sd_month = int(start_date_short[4:6])
sd_day = int(start_date_short[6:8])
sd_hour = int(start_date_short[9:11])
sd_min = int(start_date_short[11:13])
sd_sec = int(start_date_short[13:15])

fd_year = int(finish_date_short[0:4])
fd_month = int(finish_date_short[4:6])
fd_day = int(finish_date_short[6:8])
fd_hour = int(finish_date_short[9:11])
fd_min = int(finish_date_short[11:13])
fd_sec = int(finish_date_short[13:15])

In [ ]:
start = datetime.datetime(year=sd_year, month=sd_month, day=sd_day, hour=sd_hour, minute=sd_min,second=sd_sec)
end = datetime.datetime(year=fd_year, month=fd_month, day=fd_day, hour=sd_hour, minute=fd_min,second=fd_sec)
diff = end - start
print(diff)

In [ ]:
import smtplib
from email.message import EmailMessage

msg_subject = "[GG Colab] Model {}".format(start_date_short)
msg_content = "It take {} for training.".format(diff)

msg = EmailMessage()
msg['Subject'] = msg_subject
msg['From'] = '*'
msg['To']='ptmhoang97@gmail.com'
msg.set_content(msg_content)

server = smtplib.SMTP_SSL('smtp.gmail.com',465)
server.login("ptmhoang97@gmail.com","euclxykrqkkrquqz")
server.send_message(msg)
server.quit()